# 本地CPU环境测试，ONNX和TensorRT

# 安装依赖包

In [5]:
pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 56.5 MB 566 kB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [12]:
pip uninstall -y torch torchvision

Found existing installation: torch 2.2.2
Uninstalling torch-2.2.2:
  Successfully uninstalled torch-2.2.2
Found existing installation: torchvision 0.17.2
Uninstalling torchvision-0.17.2:
  Successfully uninstalled torchvision-0.17.2
Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install torch==1.10.0 torchvision==0.11.1

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 147.1 MB 643 kB/s eta 0:00:01█████████                  | 64.2 MB 600 kB/s eta 0:02:19
     |████████████████████████████████| 1.2 MB 3.6 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install onnx

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 16.5 MB 2.9 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install onnxruntime

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 15.9 MB 453 kB/s eta 0:00:01
     |████████████████████████████████| 46 kB 1.1 MB/s eta 0:00:01
     |████████████████████████████████| 86 kB 2.9 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import torch
import torch.backends.cudnn as cudnn
import numpy as np
import cv2

#忽略警告
import warnings
warnings.filterwarnings("ignore")

/Users/hepeng01/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 加载预训练模型
def load_model(model, pretrained_path, load_to_cpu):
    print('Loading pretrained model from {}'.format(pretrained_path))
    if load_to_cpu:
        pretrained_dict = torch.load(pretrained_path, map_location=lambda storage, loc: storage)
    else:
        device = torch.cuda.current_device()
        pretrained_dict = torch.load(pretrained_path, map_location=lambda storage, loc: storage.cuda(device))

    if "state_dict" in pretrained_dict.keys():
        pretrained_dict = remove_prefix(pretrained_dict['state_dict'], 'module.')
    else:
        pretrained_dict = remove_prefix(pretrained_dict, 'module.')

    # 加载权重
    model.load_state_dict(pretrained_dict, strict=False)
    return model

def remove_prefix(state_dict, prefix):
    print('remove prefix \'{}\''.format(prefix))
    f = lambda x: x.split(prefix, 1)[-1] if x.startswith(prefix) else x
    return {f(key): value for key, value in state_dict.items()}

#初始化模型

# 使用backbone网络
network = 'resnet50'  #'mobile0.25'

# 加载模型配置
cfg = cfg_re50 if network == "resnet50" else cfg_mnet
print(cfg)

# 初始化模型
net = RetinaFace(cfg=cfg, phase='test')
print(net)

#预训练模型参数
trained_weigth_path = './Resnet50_Final.pth' #mobilenet0.25_epoch_10.pth

# cpu = True #是否使用cpu
cpu = not torch.cuda.is_available() #当前环境没有gpu使用cpu
model = load_model(net, trained_weigth_path, cpu)
model.eval() #设置模型为评估模式

NameError: name 'net' is not defined

In [9]:
import torch
from itertools import product as product
import numpy as np
from math import ceil

#生成先验框
class PriorBox(object):
    def __init__(self, cfg, image_size=None, phase='train'):
        super(PriorBox, self).__init__()
        self.min_sizes = cfg['min_sizes']
        self.steps = cfg['steps']
        self.clip = cfg['clip']
        self.image_size = image_size
        self.feature_maps = [[ceil(self.image_size[0]/step), ceil(self.image_size[1]/step)] for step in self.steps]
        self.name = "s"

    def forward(self):
        anchors = []
        for k, f in enumerate(self.feature_maps):
            min_sizes = self.min_sizes[k]
            for i, j in product(range(f[0]), range(f[1])):
                for min_size in min_sizes:
                    s_kx = min_size / self.image_size[1]
                    s_ky = min_size / self.image_size[0]
                    dense_cx = [x * self.steps[k] / self.image_size[1] for x in [j + 0.5]]
                    dense_cy = [y * self.steps[k] / self.image_size[0] for y in [i + 0.5]]
                    for cy, cx in product(dense_cy, dense_cx):
                        anchors += [cx, cy, s_kx, s_ky]

        # back to torch land
        output = torch.Tensor(anchors).view(-1, 4)
        if self.clip:
            output.clamp_(max=1, min=0)
        return output


In [10]:
import numpy as np

#解码先验框
def py_cpu_nms(dets, thresh):
    """Pure Python NMS baseline."""
    x1 = dets[:, 0]
    y1 = dets[:, 1]
    x2 = dets[:, 2]
    y2 = dets[:, 3]
    scores = dets[:, 4]

    areas = (x2 - x1 + 1) * (y2 - y1 + 1)
    order = scores.argsort()[::-1]

    keep = []
    while order.size > 0:
        i = order[0]
        keep.append(i)
        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])

        w = np.maximum(0.0, xx2 - xx1 + 1)
        h = np.maximum(0.0, yy2 - yy1 + 1)
        inter = w * h
        ovr = inter / (areas[i] + areas[order[1:]] - inter)

        inds = np.where(ovr <= thresh)[0]
        order = order[inds + 1]

    return keep

In [11]:
#解码位置
def decode_loc(loc, priors, variances):
    """Decode locations from predictions using priors to undo
    the encoding we did for offset regression at train time.
    Args:
        loc (tensor): location predictions for loc layers,
            Shape: [num_priors,4]
        priors (tensor): Prior boxes in center-offset form.
            Shape: [num_priors,4].
        variances: (list[float]) Variances of priorboxes
    Return:
        decoded bounding box predictions
    """

    boxes = torch.cat((
        priors[:, :2] + loc[:, :2] * variances[0] * priors[:, 2:],
        priors[:, 2:] * torch.exp(loc[:, 2:] * variances[1])), 1)
    boxes[:, :2] -= boxes[:, 2:] / 2
    boxes[:, 2:] += boxes[:, :2]
    return boxes
#解码关键点
def decode_landm(pre, priors, variances):
    """Decode landm from predictions using priors to undo
    the encoding we did for offset regression at train time.
    Args:
        pre (tensor): landm predictions for loc layers,
            Shape: [num_priors,10]
        priors (tensor): Prior boxes in center-offset form.
            Shape: [num_priors,4].
        variances: (list[float]) Variances of priorboxes
    Return:
        decoded landm predictions
    """
    landms = torch.cat((priors[:, :2] + pre[:, :2] * variances[0] * priors[:, 2:],
                        priors[:, :2] + pre[:, 2:4] * variances[0] * priors[:, 2:],
                        priors[:, :2] + pre[:, 4:6] * variances[0] * priors[:, 2:],
                        priors[:, :2] + pre[:, 6:8] * variances[0] * priors[:, 2:],
                        priors[:, :2] + pre[:, 8:10] * variances[0] * priors[:, 2:],
                        ), dim=1)
    return landms

In [12]:
cfg_mnet = {
    'name': 'mobilenet0.25',
    'min_sizes': [[16, 32], [64, 128], [256, 512]],
    'steps': [8, 16, 32],
    'variance': [0.1, 0.2],
    'clip': False,
    'loc_weight': 2.0,
    'gpu_train': True,
    'batch_size': 32,
    'ngpu': 1,
    'epoch': 250,
    'decay1': 190,
    'decay2': 220,
    'image_size': 640,
    'pretrain': True,
    'return_layers': {'stage1': 1, 'stage2': 2, 'stage3': 3},
    'in_channel': 32,
    'out_channel': 64
}

cfg_re50 = {
    'name': 'Resnet50',
    'min_sizes': [[16, 32], [64, 128], [256, 512]],
    'steps': [8, 16, 32],
    'variance': [0.1, 0.2],
    'clip': False,
    'loc_weight': 2.0,
    'gpu_train': False,
    'batch_size': 24,
    'ngpu': 0,
    'epoch': 100,
    'decay1': 70,
    'decay2': 90,
    'image_size': 840,
    'pretrain': True,
    'return_layers': {'layer2': 1, 'layer3': 2, 'layer4': 3},
    'in_channel': 256,
    'out_channel': 256
}

# 使用backbone网络
network = 'resnet50'  #'mobile0.25'
# 加载模型配置
cfg = cfg_re50 if network == "resnet50" else cfg_mnet
print(cfg)

{'name': 'Resnet50', 'min_sizes': [[16, 32], [64, 128], [256, 512]], 'steps': [8, 16, 32], 'variance': [0.1, 0.2], 'clip': False, 'loc_weight': 2.0, 'gpu_train': False, 'batch_size': 24, 'ngpu': 0, 'epoch': 100, 'decay1': 70, 'decay2': 90, 'image_size': 840, 'pretrain': True, 'return_layers': {'layer2': 1, 'layer3': 2, 'layer4': 3}, 'in_channel': 256, 'out_channel': 256}


In [13]:
# 加载图片并预处理
def preprocess_image(image_path):
  img_raw = cv2.imread(image_path, cv2.IMREAD_COLOR)
  if img_raw is None:
    print('image cant open, check please')
    return

  img = np.float32(img_raw) #转为浮点数
  im_height, im_width, _ = img.shape #获取高，宽，忽略第三个参数通道数

  scale = torch.Tensor([img.shape[1], img.shape[0], img.shape[1], img.shape[0]])

  img -= (104, 117, 123) #减去均值进行归一化，RGB 固定104, 117, 123
  img = img.transpose(2, 0, 1) #转置图像维度 维度从 (height, width, channels) 转换为 (channels, height, width)
  img = np.expand_dims(img, axis=0) #添加批量维度 形状从 (channels, height, width) 变为 (1, channels, height, width)。

  return img, img_raw, scale


# 解码预测结果 
def decode_results(loc, conf, landms, resize, scale, img_shape):
  im_height = img_shape[2]
  im_width = img_shape[3] 

  #### 解码预测结果 #####
  priorbox = PriorBox(cfg, image_size=(im_height, im_width)) #生成先验框（prior boxes），用于帮助模型预测目标的位置
  priors = priorbox.forward() #生成先验框
  prior_data = priors.data #将先验框数据转换为 NumPy 数组 

  #解码位置预测，将模型的输出解码为实际的边框坐标 
  # boxes = decode(loc.data.squeeze(0), prior_data, cfg['variance']) # （修改点） 使用先验框和模型的输出（loc[0]）解码出实际的边框坐标 cfg['variance'] 是解码过程中使用的方差。
  boxes = decode_loc(torch.tensor(loc[0]), torch.tensor(prior_data), cfg['variance']) #使用先验框和模型的输出（loc[0]）解码出实际的边框坐标。cfg['variance'] 是解码过程中使用的方差。
  boxes = boxes * scale / resize #对解码后的边框进行缩放，使其与原图的大小一致。scale 是之前定义的缩放比例张量，resize 是缩放因子，这里为1。
  boxes = boxes.cpu().numpy() #将解码后的边框转换为 NumPy 数组

  #提取置信度 
  # scores = conf.squeeze(0).data.cpu().numpy()[:, 1] #提取了每个预测框的置信度分数 （修改点）
  scores = conf[0][:, 1] #提取了每个预测框的置信度分数

  #解码关键点预测
  # landms = decode_landm(landms.data.squeeze(0), prior_data, cfg['variance']) （修改点）
  landms = decode_landm(torch.tensor(landms[0]), torch.tensor(prior_data), cfg['variance'])

  scale1 = torch.Tensor([img_shape[3], img_shape[2], img_shape[3], img_shape[2],
                        img_shape[3], img_shape[2], img_shape[3], img_shape[2],
                        img_shape[3], img_shape[2]])
  landms = landms * scale1 / resize
  landms = landms.cpu().numpy()

  return boxes, scores, landms

# 过滤检测框
def filter_boxes(boxes, scores, landms, confidence_threshold, nms_threshold):
  # 过滤低置信度检测结果
  inds = np.where(scores > confidence_threshold)[0]
  boxes = boxes[inds] #保留置信度高于阈值的边框
  landms = landms[inds] #保留置信度高于阈值的关键点
  scores = scores[inds] #保留置信度高于阈值的置信度分数

  # 按置信度排序
  order = scores.argsort()[::-1] #[::-1] 表示从高到低排序。
  boxes = boxes[order] #按排序后的索引重新排列边框
  landms = landms[order] #按排序后的索引重新排列关键点
  scores = scores[order] #按排序后的索引重新排列置信度分数

  # 将边框和置信度分数水平堆叠在一起，形成一个新的数组 dets。scores[:, np.newaxis] 将置信度分数转换为列向量。
  dets = np.hstack((boxes, scores[:, np.newaxis], landms)).astype(np.float32, copy=False)

  # Apply NMS
  # 使用 NMS 算法去除重叠的检测框。nms_threshold 是 NMS 的阈值，py_cpu_nms 是一个实现 NMS 的函数。keep 是保留下来的检测框的索引。
  keep = py_cpu_nms(dets[:, :5], nms_threshold)
  dets = dets[keep, :] #保留 NMS 后的检测框
  landms = landms[keep] #保留 NMS 后的关键点

  return dets

# 在图上绘制检测结果
def draw_detection(image, dets, save_path, vis_thres=0.6):
  if dets is None:
     return
  # 绘制检测结果（可能多张人脸）
  for i, b in enumerate(dets):
      if b[4] < vis_thres:
          continue
      text = "{:.4f}".format(b[4])
      b = list(map(int, b))
      cv2.rectangle(image, (b[0], b[1]), (b[2], b[3]), (0, 0, 255), 2)
      
      cx = b[0]
      cy = b[1] + 12
      cv2.putText(image, text, (cx, cy),
                  cv2.FONT_HERSHEY_DUPLEX, 0.5, (255, 255, 255))

      # 绘制关键点
      cv2.circle(image, (b[5], b[6]), 1, (0, 0, 255), 4)
      cv2.circle(image, (b[7], b[8]), 1, (0, 255, 255), 4)
      cv2.circle(image, (b[9], b[10]), 1, (255, 0, 255), 4)
      cv2.circle(image, (b[11], b[12]), 1, (0, 255, 0), 4)
      cv2.circle(image, (b[13], b[14]), 1, (255, 0, 0), 4)
      
      print(f"Box {i}: {b}")
  # 保存结果图像
  cv2.imwrite(save_path, image)
  print(f"Detection result saved to {save_path}")

  # 展示结果图像
  # from google.colab.patches import cv2_imshow
  # cv2_imshow(img_raw)

# 图片保存路径
def get_save_path(image_path, suffix="_new"):
  directory, filename = os.path.split(image_path)
  name, ext = os.path.splitext(filename)
  save_path = os.path.join(directory, f"{name}{suffix}{ext}")
  return save_path


# 使用ONNXRuntime

In [19]:
import onnxruntime as ort
# 加载 ONNX 模型
onnx_model_path = "model_repository/kavin_face_model/1/model.onnx"
ort_session = ort.InferenceSession(onnx_model_path)

# 执行ONNX模型推理
def inference_onnx(ort_session, img):
  # 前向传播
  input_name = ort_session.get_inputs()[0].name #获取输入名称
  loc, conf, landms = ort_session.run(None, {input_name: img}) #使用 ONNX Runtime 进行模型推理，第一个参数 None 表示我们要获取模型的所有输出，第二个参数是一个字典，键是输入名称，值是输入数据。

  # loc：位置预测（bounding box location predictions）
  # conf：置信度（confidence scores）
  # landms：关键点（landmark predictions）
  print("loc shape:", loc.shape)
  print("conf shape:", conf.shape)
  print("landms shape:", landms.shape)  
  return loc, conf, landms

#人脸检测,使用ONNX模型
def check_face_use_onnx(ort_session, image_path):
  confidence_threshold = 0.5 # 置信度阈值，过滤低置信度检测结果
  nms_threshold = 0.4 # NMS 阈值，过滤重叠的检测框
  vis_thres = 0.6 # 可视化阈值，只绘制置信度高于该阈值的检测结果
  resize = 1 # 使用原图

  # 加载图片并预处理
  img, img_raw, scale = preprocess_image(image_path)

  # 执行ONNX模型推理
  loc, conf, landms = inference_onnx(ort_session, img)
  
  # 推理结果解码
  #img.shape 表示模型输入的图像的形状。(1, 3, 308, 450)  
  boxes, scores, landms = decode_results(loc, conf, landms, resize, scale, img.shape)

  # 过滤检测框
  dets = filter_boxes(boxes, scores, landms, confidence_threshold, nms_threshold)

  # 在图像上结果标注并保存
  save_path = get_save_path(image_path, "_onnx") 
  draw_detection(img_raw, dets, save_path, vis_thres)

# 检测人脸
check_face_use_onnx(ort_session, "test/1.png")

loc shape: (1, 16952, 4)
conf shape: (1, 16952, 2)
landms shape: (1, 16952, 10)
Box 0: [275, 0, 469, 266, 0, 288, 92, 366, 97, 297, 148, 292, 199, 347, 204]
Detection result saved to test/1_onnx.png


# 使用Docker API

In [20]:
# 使用接口推理
import requests
def inference_api(img):
  # 构造推理请求的JSON结构
  infer_request = {
        "inputs": [
            {
                "name": "input",
                "shape": img.shape,
                "datatype": "FP32",
                "data": img.flatten().tolist()
            }
        ]
  }  
  # 发送推理请求
  try:
    url = 'http://localhost:8000/v2/models/kavin_face_model/infer'
    response = requests.post(url, json=infer_request)
    result = response.json()
    if response.status_code != 200:
      raise Exception(result)
  except Exception as e:
    print("Error in inference request:", e)
    return

  # 解析推理结果
  conf = np.array(result['outputs'][0]['data']).reshape(result['outputs'][0]['shape'])
  landms = np.array(result['outputs'][1]['data']).reshape(result['outputs'][1]['shape'])
  loc = np.array(result['outputs'][2]['data']).reshape(result['outputs'][2]['shape'])

  # 打印输出的形状
  print("loc shape:", loc.shape)
  print("conf shape:", conf.shape)
  print("landms shape:", landms.shape)
  return loc, conf, landms

#人脸检测,使用API
def check_face_use_api(image_path):
  confidence_threshold = 0.5
  nms_threshold = 0.4
  vis_thres = 0.6
  resize = 1 # 使用原图

  # 加载图片并预处理
  img, img_raw, scale = preprocess_image(image_path)

  # 执行API推理
  loc, conf, landms = inference_api(img) 

  # 推理结果解码
  #img.shape 表示模型输入的图像的形状。(1, 3, 308, 450)  
  boxes, scores, landms = decode_results(loc, conf, landms, resize, scale, img.shape)

  # 过滤检测框
  dets = filter_boxes(boxes, scores, landms, confidence_threshold, nms_threshold)

  # 在图像上结果标注并保存
  save_path = get_save_path(image_path, "_api") 
  draw_detection(img_raw, dets, save_path, vis_thres)

check_face_use_api("test/2.jpg")

loc shape: (1, 68432, 4)
conf shape: (1, 68432, 2)
landms shape: (1, 68432, 10)
Box 0: [1458, 240, 1518, 320, 0, 1475, 271, 1502, 271, 1488, 287, 1475, 297, 1500, 296]
Box 1: [700, 225, 758, 305, 0, 715, 257, 742, 257, 729, 274, 716, 283, 741, 282]
Box 2: [1073, 232, 1132, 308, 0, 1089, 266, 1116, 264, 1104, 282, 1092, 291, 1116, 289]
Box 3: [360, 242, 417, 320, 0, 384, 273, 409, 275, 400, 292, 383, 302, 401, 303]
Detection result saved to test/2_api.jpg
